<a href="https://colab.research.google.com/github/Jaki-01/Bachelorarbeit-Ablagebereich-/blob/main/Modell_Wagnis_Erkennung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate -U
!pip install transformers --upgrade
!pip install datasets~=2.14.0
!pip install optuna~=3.3.0

In [4]:
### Beschreibung
# Beachte, dass große Teile dieses Code aus der Schlüsselquelle stammen
# Schlüsselquelle: https://github.com/MoritzLaurer/summer-school-transformers-2023/blob/main/3_tune_bert.ipynb
# Trainiertes Modell: https://huggingface.co/Jaki01/vagueness-detection-large -> Zum heru
# Verwendeter Datensatz: HIIIIIER DATENSAAATTZZZ ANGEBEN (siehe. https://github.com/Jaki-01/Bachelorarbeit-Ablagebereich-)

from google.colab import files

# Oeffnet einen Dialog zur Auswahl der Datei(en) von deinem lokalen Desktop
print("Hier die Datei GITHUB-VERLINKUNG einfügen")
uploaded = files.upload()


import pandas as pd
import numpy as np
import torch
import datasets
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    logging,
    pipeline
)
from sklearn.metrics import (
    balanced_accuracy_score,
    precision_recall_fscore_support,
    accuracy_score,
    classification_report
)
import warnings

# Ramdom Seed setzen, damit das Ergebnis reproduzierbar ist - Zahl ist beliebig
SEED_GLOBAL = 42
np.random.seed(SEED_GLOBAL)

# Import der Datein
data = pd.read_csv(r"test_file.csv", sep=";", encoding='unicode_escape')

# Bereinigen der Spaltennamenvon Leerzeichen - behebt ein technisches Problem in Bezug auf die CSV-Datei
data.columns = data.columns.str.strip()

print(data.columns)

# ueberschriften werden definiert
data = data[["label", "label_text", "text"]]

# train-test Split
df_train, df_test = train_test_split(data, test_size=0.2, random_state=SEED_GLOBAL)

# Modell laden
model_name = "bert-large-uncased"  # weitere Moegliche Modelle microsoft/deberta-v3-large, FacebookAI/roberta-large, nghuyong/ernie-2.0-large-en uncased = alles wird in Kleinsbuchstaben transformiert

# tokenizer laden
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, model_max_length=512)

# Zahlenwerte (1 und 0) in Verbindung mit den Labeln (vague, not vague) bringen
label_text = np.sort(df_test.label_text.unique()).tolist() # print -> ['not vague', 'vague']
label2id = dict(zip(np.sort(label_text), np.sort(pd.factorize(label_text, sort=True)[0]).tolist())) # print -> {'not vague': 0, 'vague': 1}
id2label = dict(zip(np.sort(pd.factorize(label_text, sort=True)[0]).tolist(), np.sort(label_text))) # print ->  {0: 'not vague', 1: 'vague'}

# Konfigurator stellt das Bert Modell ein, damit es für entsprechende Aufgaben verwendet werden kann
config = AutoConfig.from_pretrained(model_name, label2id=label2id, id2label=id2label, num_labels=len(label2id)); # print BertConfig {... usw

# Modell laden mit config
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config, ignore_mismatched_sizes=True);

# Pandas DataFrames wird in ein Hugging Face Dataset-Objekt umwandeln, um die Vorverarbeitung zu erleichtern.
dataset = datasets.DatasetDict({
    "train": datasets.Dataset.from_pandas(df_train),
    "test": datasets.Dataset.from_pandas(df_test)
})

# tokenizer
def tokenize(examples):
  return tokenizer(examples["text"], truncation=True, max_length=512)  # laenge kann auf 256 reduziert werden, aber dann wird langer Text abgeschnitten

dataset = dataset.map(tokenize, batched=True)

# entfert uerberfluessige Spalten
dataset = dataset.remove_columns(['label_text'])

# Erstelle ein directoy um ein fine-tunde Modell und trainings logs abzuspeichern.

training_directory = "Vagueness_detection" # Name des abgespeicherten Modells

# Uebersicht zu den verschiedenen Traingsparametern: https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments
train_args = TrainingArguments(
    num_train_epochs=4,  # ein guter Wert liegt zwischen 3 und 20, erhöht aber die Trainingszeit
    learning_rate=8e-5,
    per_device_train_batch_size=16,  # Höhere Werte senken die Trainingszeit, aber erhöhen die Speicheranforderungen. Optimale Werte sind 2^x Werte
    per_device_eval_batch_size=64,  # reduzieren wenn ein "out of memory" Fehler kommt
    logging_steps=20, # logging wird alle 20 Schritte durchgeführt (defaut sind 500)
    warmup_ratio=0.03,  # Gibt an wie viel vergessen werden soll bei jedem Durchgang (praevention gegen Ueberanpassung), 0,06 ist ein guter Wert für das BERT-base-model. Wenn vorheriges Wissen wiederverwendet werden soll, Wert höher setzen
    weight_decay=0.1, # verringert das Gewicht von großen Gewichten und ist eine Praeventionsmaßnahme gegen Ueberanpassung
    seed=SEED_GLOBAL, # Damit bei durchlauf des identischen Modells das gleiche Modell reproduziert wird
    load_best_model_at_end=True, # Das beste Modell wird ausgewaehlt
    metric_for_best_model="f1_macro", # Dieser Wert bestimmt, welches Modell als "das Beste" definiert ist # "f1_macro" ist die Zentrale groeße
    evaluation_strategy="epoch", # steltlt sicher, dass regelmaeßig die Qualitaet des Modells ueberprüft wird
    save_strategy = "epoch",
    report_to="all",
    output_dir=f'./results/{training_directory}',
    logging_dir=f'./logs/{training_directory}',
)

# Funktion um die Metrics zu berechnen
# Quelle: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

def compute_metrics_standard(eval_pred):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")

        labels = eval_pred.label_ids
        pred_logits = eval_pred.predictions
        preds_max = np.argmax(pred_logits, axis=1)  # argmax in jeder Zeile (Achse=1) in dem Tensor

        # metrics
        roc_auc = roc_auc_score(labels, preds_max) # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds_max, average='macro')  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
        acc_balanced = balanced_accuracy_score(labels, preds_max)
        acc_not_balanced = accuracy_score(labels, preds_max)

        metrics = {
            'accuracy': acc_not_balanced,
            'f1_macro': f1_macro,
            'accuracy_balanced': acc_balanced,
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
        }

        return metrics

# training
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=train_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics_standard
)

trainer.train()

# Evaluierung des Modells auf den Testdatensatz
results = trainer.evaluate()
print(results)


Hier die Datei GITHUB-VERLINKUNG einfügen


Saving test_file.csv to test_file.csv
Index(['label', 'label_text', 'text'], dtype='object')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/411 [00:00<?, ? examples/s]

Map:   0%|          | 0/103 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Accuracy Balanced,F1 Micro,Precision Macro,Recall Macro,Precision Micro,Recall Micro,Roc Auc
1,0.492700,0.611227,0.728155,0.421348,0.500000,0.728155,0.364078,0.500000,0.728155,0.728155,0.500000
2,0.619800,0.585564,0.728155,0.421348,0.500000,0.728155,0.364078,0.500000,0.728155,0.728155,0.500000
3,0.536600,0.618083,0.728155,0.421348,0.500000,0.728155,0.364078,0.500000,0.728155,0.728155,0.500000
4,0.555700,0.609231,0.728155,0.421348,0.500000,0.728155,0.364078,0.500000,0.728155,0.728155,0.500000


{'eval_loss': 0.6112270355224609, 'eval_accuracy': 0.7281553398058253, 'eval_f1_macro': 0.42134831460674155, 'eval_accuracy_balanced': 0.5, 'eval_f1_micro': 0.7281553398058251, 'eval_precision_macro': 0.3640776699029126, 'eval_recall_macro': 0.5, 'eval_precision_micro': 0.7281553398058253, 'eval_recall_micro': 0.7281553398058253, 'eval_roc_auc': 0.5, 'eval_runtime': 0.2881, 'eval_samples_per_second': 357.494, 'eval_steps_per_second': 6.942, 'epoch': 4.0}


In [ ]:
## Google Drive mit Google Account verbinden
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=False)
#drive.flush_and_unmount()

# insert the path where you want to save the model
os.chdir('/content/drive/MyDrive/')
print(os.getcwd())


In [ ]:
### das beste Modell im Google Drive Speichern
training_directory = "Jaki01"
model_name = "vagueness-detection"

directory_save_model = f"{training_directory}/"
model_name_custom = f"{model_name.split('/')[-1]}"
mode_custom_path = directory_save_model + model_name_custom
print(mode_custom_path)

# Zeile nur freischalten, wenn das Modell auf Google Drive gespeichert werden soll
#trainer.save_model(output_dir=mode_custom_path)